# Molecular associations with histological subtype

For now, analyze only mRNA, miRNA and RPPA.

In [8]:
from pathlib import Path

import numpy as np
import pandas as pd
import re

from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
CLINICAL_DATA_PATH_1 = "../data/broad_tcga/clinical/gdac.broadinstitute.org_STAD.Clinical_Pick_Tier1.Level_4.2016012800.0.0.tar.gz"

RPPA_PATH = "../data/broad_tcga/rppa/gdac.broadinstitute.org_STAD.RPPA_AnnotateWithGene.Level_3.2016012800.0.0/STAD.rppa.txt"
METH_PATH = "../data/broad_tcga/methylation/gdac.broadinstitute.org_STAD.Merge_methylation__humanmethylation450__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data.Level_3.2016012800.0.0.tar.gz"
MIRNA_PATH_HISEQ = "../data/broad_tcga/miRSeq/gdac.broadinstitute.org_STAD.Merge_mirnaseq__illuminahiseq_mirnaseq__bcgsc_ca__Level_3__miR_gene_expression__data.Level_3.2016012800.0.0.tar.gz"

RNA_PATH_HISEQ_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminahiseq_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_GA_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminaga_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_HISEQ_v2 = ''
RNA_PATH_HISEQ_normalized_v2 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'

In [11]:
def load_rppa(path=RPPA_PATH) -> pd.DataFrame:
    df = pd.read_csv(path, sep='\t', index_col=0, header=0)
    return df

def load_mirna(path=MIRNA_PATH_HISEQ) -> pd.DataFrame:
    df = pd.read_csv(path, sep='\t', header=0, index_col=0)
    df = df.iloc[1:, (df.loc['miRNA_ID'] == 'reads_per_million_miRNA_mapped').values]
    return df

def load_mrna(path=RNA_PATH_HISEQ_normalized_v2) -> pd.DataFrame:
    df = pd.read_csv(path, compression='gzip',
                    index_col=0, header=0, sep='\t').iloc[1:, :]
    return df

def load_clinical(path=CLINICAL_DATA_PATH_1) -> pd.DataFrame:
    df = pd.read_csv(path, index_col=0, sep='\t')
    return df

def load_data(types: list = ['clinical', 'mRNA', 'miRNA', 'rppa']) -> dict:
    '''
    Loads dataframes into dict by data type.
    
    :param types
    :return dict
    '''
    
    # mutations seems big file, might take a while
    # methylation is with similar issue
    
    types_l = [s.lower() for s in types]  # lower strings' chars
    d = {}
    d['clinical'] = load_clinical()  # need to load clinical anyways
    
    if 'mrna' in types_l:
        d['mrna'] = load_mrna()
    if 'mirna' in types_l:
        d['mirna'] = load_mirna()
    if 'rppa' in types_l:
        d['rppa'] = load_rppa()
    
    return d

In [4]:
def get_ids_by_subtype(df: pd.DataFrame, subtype: str or list):
    '''
    '''
    
    histotypes = df.loc['histological_type'].iloc[0].to_list()
    
    short_names = {'src': r'signet ring type',
                  'diffuse': r'stomach, adenocarcinoma, diffuse type'}
    
    if type(subtype) == list:
        pass
    
    elif type(subtype) == str:
        assert subtype in short_names.keys()
        subtype = [subtype]
    
    else:
        raise ValueError("subtype string not recognized.")
    
    ids = []
    for subt in subtype:
        assert subt in short_names.keys()
        
        pattern = short_names[subt]
        subt_ids = df.columns[list(map(
            lambda x: bool(re.search(pattern, str(x))),
            histotypes))].to_list()
        ids += subt_ids
            
    return ids


def get_src_mask(df: pd.DataFrame, ids: list):
    '''
    Gets the boolean mask for samples corresponding to ids.
    
    :param df - dataframe in expected format (rows are omics' features, columns are tcga samples)
    :param ids - list of tcga ids of interest. E.g. ids of patients with SRC.
    '''
    rgx = r"|".join(ids)
    src_mask = list(map(lambda x: bool(re.search(rgx, x)),
                        map(lambda x: "-".join(x.lower().split("-")[:3]), df.columns)))
    return src_mask


# Perform Mann-Whitney U-test - nonparametric test for testing different expression for signet ring cells
# An equivalent to ANOVA test is Kruskal-Wallis one way analysis, which was employed by broad gdac,
# as a nonparametric, robust equivalent to one-way anova.
# Let's have a function that takes a dataframe and computes this test for histologic type, in 2 versions:
# kruskal-wallis and mann-whitney for src (with direction).

def associations_with_src(df: pd.DataFrame, src_ids: list, background_ids: list or None) -> dict:
    '''
    Computes dict of associations (test statistic and p-value) 
     of features in df with sample appartenance in src_ids.
    
    :param df
    :param src_ids
    '''
    
    src_mask = get_src_mask(df, src_ids)
    
    if background_ids is None:
        nonsrc_mask = [not i for i in src_mask]
    elif type(background_ids) == list:
        nonsrc_mask = get_src_mask(df, background_ids)
    else:
        raise TypeError("background_ids type not recognized.")
    
    res = {}
    for idx, row in df.iterrows():
        src_vals    = row[src_mask].astype('float64')
        nonsrc_vals = row[nonsrc_mask].astype('float64')
        
        try:
            mwu = mannwhitneyu(nonsrc_vals, src_vals, alternative='two-sided')
        except ValueError:
            # assume all nrs are 0 and equal
            res[idx] = [0, 1]
        
        mean_diff   = np.mean(src_vals) - np.mean(nonsrc_vals)
        median_diff = np.median(src_vals) - np.median(nonsrc_vals)
        
        res[idx] = [mean_diff, median_diff, mwu[0], mwu[1]]
    
    return res


def compute_src_statistics(df: pd.DataFrame, src_ids: list, background_ids: list or None) -> pd.DataFrame:
    '''
    Computes statistics for associating molecular features of SRC subtype.
    
    :param df - samples are tcga ids of all patients in background
    :param src_ids - list of ids of samples corresponding to patients with SRC tumors.
    '''
    
    r = associations_with_src(df, src_ids, background_ids)
    r = pd.DataFrame.from_dict(r, orient='columns').transpose()
    r.columns = ['mean_diff', 'median_diff', 'test-statistic', 'p-value']
    r.sort_values(['p-value'], ascending=True, inplace=True)
    
    # Add multiple test correction
    mtcorr = multipletests(r['p-value'],
                          alpha=0.05,
                          method='fdr_bh')
    r['p-value_corrected'] = mtcorr[1]
    r['p-value_reject']    = mtcorr[0]
    
    return r


def compute_allomics_src_statistics(target: str or list, 
                                   background: str or list or None = None,
                                   types: list = ['mrna', 'mirna']) -> pd.DataFrame:
    '''
    Computes statistics of association with SRC subtype for different multiomics data.
    
    :param paths - dict of omics_type -> Path()
    :param src_ids - list of ids of src type
    
    :return df with columns [feature, omics_type, test-statistic, p-value, p-value-corrected, p-value-reject]
    '''
    
    ### Load data
    dfs = load_data(types)
    
    ### Get ids for target and background groups for comparison
    src_ids = get_ids_by_subtype(dfs['clinical'], subtype=target)
    
    if background is None:
        background_ids = None
    else:
        background_ids = get_ids_by_subtype(dfs['clinical'], subtype=background)
    
    ### Compute results and collect
    res = {}
    for omics_type, df in dfs.items():
        
        if omics_type == 'clinical':
            continue
            
        df_res = compute_src_statistics(df, src_ids, background_ids)
        df_res['data_type'] = omics_type
        res[omics_type] = df_res
    
    ### Aggregate results in single dataframe
    df_final = pd.concat(res.values(), axis=0)
    df_final.sort_values(['p-value_corrected'], ascending=True, inplace=True)
    
    return df_final

In [13]:
# TODOS:
# - compare across various subtypes
# - add logistic regression to account for covariates (which ones ?...)  -  optional, showing off statistics.

def run_experiment(results_dir='results'):
    
    results_path = Path(results_dir)
    if not results_path.is_dir():
        results_path.mkdir()

    savefp = results_path / 'associations_src_vs_diffuse.csv'

    target = ['src']
    background = ['diffuse']
    datatypes = ['mrna', 'mirna', 'rppa'] #'mirna',  # to add: cnv, mutations, methylation?

    ### Currently it ignores the covariates, might add with a multiple logistic regression.
    res = compute_allomics_src_statistics(target, background, datatypes)
    res.to_csv(savefp, header=True, index=True)
    
    return res

def load_results(path='results/associations_src_vs_diffuse.csv'):
    df = pd.read_csv(path, header=0, index_col=0)
    return df

In [14]:
res = load_results()

In [30]:
print(res.iloc[:2,:3].to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  mean\_diff &  median\_diff &  test-statistic \\
\midrule
LOC100271831|100271831 &   8.844388 &          0.0 &           266.0 \\
CABP2|51475            &   0.148125 &          0.0 &           300.0 \\
\bottomrule
\end{tabular}



\\begin{tabular}{lrrrrrll}\n\\toprule\n{} &  mean\\_diff &  median\\_diff &  test-statistic &       p-value &  p-value\\_corrected &  p-value\\_reject & data\\_type \\\\\n\\midrule\nLOC100271831|100271831 &   8.844388 &          0.0 &           266.0 &  2.720926e-07 &           0.004066 &            True &      mrna \\\\\nCABP2|51475            &   0.148125 &          0.0 &           300.0 &  3.960660e-07 &           0.004066 &            True &      mrna \\\\\nNA|DIRAS3              &  -0.066495 &          NaN &           453.0 &  2.293746e-04 &           0.014909 &            True &      rppa \\\\\nMAPK14|p38             &   0.013780 &          NaN &           456.0 &  1.828017e-04 &           0.014909 &            True &      rppa \\\\\nXBP1|XBP1              &  -0.248150 &          NaN &           461.0 &  1.243618e-04 &           0.014909 &            True &      rppa \\\\\n\\bottomrule\n\\end{tabular}\n